In [1]:
import yfinance as yf
import datetime as dt
from pandas.tseries.offsets import MonthEnd
from pandas_datareader import data as pdr
from nsetools import Nse
from nsepython import *
import pandas as pd


import yfinance as yf
import datetime as dt
from pandas.tseries.offsets import MonthEnd
from pandas_datareader import data as pdr
from nsetools import Nse
from nsepython import *
import pandas as pd


def calculate_ROC(start_date, indexToFetchConstituent='NIFTY%20500'):
    constituentData = nsefetch('https://www.nseindia.com/api/equity-stockIndices?index='+indexToFetchConstituent)
    symbolList = [individalScrip['symbol']+'.NS' for individalScrip in constituentData['data']]

    end = dt.datetime.now()
    start = dt.datetime.strptime(start_date, '%Y-%m-%d')
    input_stocks = symbolList[:51]

    yf.pdr_override()
    df_stocks = pdr.get_data_yahoo(input_stocks, start, end)['Close']
    df_ROC = df_stocks.pct_change()

    return df_ROC
start_date = input("Enter start date in YYYY-MM-DD format: ")
df_ROC = calculate_ROC(start_date)

Enter start date in YYYY-MM-DD format: 2015-01-01
[*********************100%***********************]  51 of 51 completed

1 Failed download:
- NIFTY 500.NS: No timezone found, symbol may be delisted


In [2]:
def getting_inputs():
    ref_date=input("enter the ref_date")
    n=int(input("enter the value of n"))
    return ref_date, n

ref_date, n = getting_inputs()

def winner(ref_date, n):
    ref_date=pd.to_datetime(ref_date)
    min_ref_date=df_ROC[df_ROC.index <=ref_date].tail(n).head(1).index[0]
    df1_ROC=df_ROC[min_ref_date:ref_date]
    df1_ROC=df1_ROC[(df1_ROC[df1_ROC.columns] > 0)]
    df1_ROC=df1_ROC[df1_ROC.columns[~df1_ROC.isnull().any()]]
    df1_ROC_rank=df1_ROC.sum(axis=0).sort_values(ascending=False)
    df1_ROC_rank=df1_ROC_rank.to_frame()
    df1_ROC_rank.insert(1,"rank",range(1,len(df1_ROC_rank)+1))
    winner_df=df1_ROC_rank
    winner_df.rename({0: 'ROC'}, axis=1, inplace=True)
    return min_ref_date,winner_df

def next_day_ROC_winner(min_ref_date,ref_date,winner_df):
    ref_date=pd.to_datetime(ref_date)
    df1_ROC=df_ROC[min_ref_date:ref_date]
    winner_index=winner_df.index
    next_day_ROC_winner=df_ROC[df_ROC.index >ref_date].head(1)
    l=list(next_day_ROC_winner[winner_index].iloc[0])
    next_date=next_day_ROC_winner[winner_index].reset_index()['Date'][0]
    winner_df.insert(2,"next_day_ROC_winner",l,True)
    return winner_df,next_date



def loser(ref_date,n):
    ref_date=pd.to_datetime(ref_date)
    min_ref_date=df_ROC[df_ROC.index <=ref_date].tail(n).head(1).index[0]
    df1_ROC=df_ROC[min_ref_date:ref_date]
    df1_ROC=df1_ROC[(df1_ROC[df1_ROC.columns] < 0)]
    df1_ROC=df1_ROC[df1_ROC.columns[~df1_ROC.isnull().any()]]
    df1_ROC_rank=df1_ROC.sum(axis=0).sort_values(ascending=True)
    df1_ROC_rank=df1_ROC_rank.to_frame()
    df1_ROC_rank.insert(1,"rank",range(1,len(df1_ROC_rank)+1))
    loser_df=df1_ROC_rank
    loser_df.rename({0: 'ROC'}, axis=1, inplace=True)
    return min_ref_date,loser_df





def next_day_ROC_loser(min_ref_date,ref_date,loser_df):
    ref_date=pd.to_datetime(ref_date)
    df1_ROC=df_ROC[min_ref_date:ref_date]
    loser_index=loser_df.index
    next_day_ROC_loser=df_ROC[df_ROC.index >ref_date].head(1)
    l=list(next_day_ROC_loser[loser_index].iloc[0])
    loser_df.insert(2,"next_day_ROC_loser",l,True)
    return loser_df




def score(winner_df,loser_df,next_date):
    new_df=pd.DataFrame()
    winner_success = len(winner_df.loc[winner_df['next_day_ROC_winner'] > 0])
    winner_fail = len(winner_df.loc[winner_df['next_day_ROC_winner'] < 0])
    loser_success = len(loser_df.loc[loser_df['next_day_ROC_loser'] < 0])
    loser_fail = len(loser_df.loc[loser_df['next_day_ROC_loser'] > 0])
    #score=success-fail
    data1={'Date':[next_date],'winner_success':winner_success,'winner_fail':winner_fail,
       'loser_success':loser_success,'loser_fail':loser_fail}
    new_df=pd.DataFrame(data1)
    return new_df





def calculate_score(ref_date, n):
    min_ref_date,winner_df = winner(ref_date, n)
    winner_df,next_date = next_day_ROC_winner(min_ref_date,ref_date,winner_df)
    min_ref_date,loser_df = loser(ref_date, n)
    loser_df = next_day_ROC_loser(min_ref_date,ref_date,loser_df)

    return score(winner_df,loser_df,next_date)
print(calculate_score(ref_date, n))










enter the ref_date2021-4-5
enter the value of n5
        Date  winner_success  winner_fail  loser_success  loser_fail
0 2021-04-06               0            0              0           0


In [3]:
def score_multiple_dates():
    end_date = input("Enter the end date in YYYY-MM-DD format: ")
    n = int(input("Enter the value of n: "))
    year, month, day = map(int, end_date.split('-'))
    end_day = dt.date(year, month, day)

    start_day = end_day - dt.timedelta(days=n)  # calculate start date based on end date and n
    
    ref_dates = []  # store dates in sequence 
    for i in range(n):
        ref_dates.append(start_day + dt.timedelta(days=i))
    
    for i in ref_dates:
        print(calculate_score(i, n))

score_multiple_dates()



Enter the end date in YYYY-MM-DD format: 2021-04-8
Enter the value of n: 5
        Date  winner_success  winner_fail  loser_success  loser_fail
0 2021-04-05               0            1              0           0
        Date  winner_success  winner_fail  loser_success  loser_fail
0 2021-04-05               0            1              0           0
        Date  winner_success  winner_fail  loser_success  loser_fail
0 2021-04-06               0            0              0           0
        Date  winner_success  winner_fail  loser_success  loser_fail
0 2021-04-07               0            0              0           0
        Date  winner_success  winner_fail  loser_success  loser_fail
0 2021-04-08               0            1              0           0
